In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

# Define the Pose Estimation Model
class PoseEstimationNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(PoseEstimationNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(512, output_size)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

class ObjectDatasetByID(Dataset):
    def __init__(self, data_2d_path, data_3d_path):
        self.data_by_id = defaultdict(lambda: {"2d": [], "3d": []})
        self.expected_output_size = 31  # Expected output size based on structure

        files_2d = sorted([f for f in os.listdir(data_2d_path) if f.endswith('.csv')])
        
        for file_2d in files_2d:
            data_2d_file_path = os.path.join(data_2d_path, file_2d)
            data_3d_file_path = os.path.join(data_3d_path, file_2d)

            if not os.path.exists(data_3d_file_path):
                print(f"Skipping; 3D file missing for: {file_2d}")
                continue

            try:
                df_2d = pd.read_csv(data_2d_file_path, header=None)
                df_3d = pd.read_csv(data_3d_file_path, header=None)

                if len(df_2d) != len(df_3d):
                    print(f"Skipping; row count mismatch in files: {file_2d}")
                    continue
                if df_3d.shape[1] != self.expected_output_size:
                    print(f"Skipping; 3D data size mismatch for file {file_2d}")
                    continue

                for row_2d, row_3d in zip(df_2d.values, df_3d.values):
                    obj_id = int(row_2d[0])  # First column is the ID
                    self.data_by_id[obj_id]["2d"].append(torch.tensor(row_2d[1:], dtype=torch.float32)) 
                    self.data_by_id[obj_id]["3d"].append(torch.tensor(row_3d[1:], dtype=torch.float32))

            except (pd.errors.EmptyDataError, ValueError):
                print(f"Skipping invalid file pair: {file_2d}")

    def get_data_by_id(self):
        # Returns the data separated by ID
        return {obj_id: (torch.stack(data["2d"]), torch.stack(data["3d"]))
                for obj_id, data in self.data_by_id.items()}


In [9]:
import torch.optim as optim
import time

# Evaluation function
def evaluate_model(model, data_loader, criterion, device='cpu'):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_loss += loss.item()
    return running_loss / len(data_loader)

# Training function with early stopping
def train_model_for_id(
    model, train_loader, val_loader, criterion, optimizer, 
    num_epochs=10, patience=5, device='cpu', obj_id=None
):
    model.to(device)
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    print(f"\n{'='*50}")
    print(f"Training Start for ID {obj_id}")
    print(f"{'='*50}\n")

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            # Handle single-sample batches
            if inputs.size(0) < 2:
                print("Switching to eval mode for single-sample batch.")
                model.eval()  # Use eval mode for BatchNorm1d
                with torch.no_grad():
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                model.train()  # Switch back to train mode
                continue

            # Normal training process for batches with sufficient samples
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()

        train_loss = running_loss / len(train_loader)
        val_loss = evaluate_model(model, val_loader, criterion, device)

        print(f"ID: {obj_id} - Epoch [{epoch+1}/{num_epochs}] - "
              f"Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

        # Early stopping and model saving logic
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
            torch.save(model.state_dict(), f"sye{obj_id}.pth")
            print(f"Validation loss improved. Model for ID {obj_id} saved.")
        else:
            epochs_without_improvement += 1
            print(f"No improvement for {epochs_without_improvement} epochs.")

        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered for ID {obj_id} after {epoch+1} epochs.")
            break


In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

base_path = "C:\\Users\\sakar\\OneDrive\\mt-datas\\synthetic_data"
dataset_name = "8_correct_relative"
data_2d_subfolder = "2d_data"
data_3d_subfolder = "3d_data"

# Construct paths to the data directories
data_2d_path = os.path.join(base_path, dataset_name, data_2d_subfolder)
data_3d_path = os.path.join(base_path, dataset_name, data_3d_subfolder)

# Load the dataset and get data by ID
object_dataset_by_id = ObjectDatasetByID(data_2d_path, data_3d_path)
data_by_id = object_dataset_by_id.get_data_by_id()

# Model parameters
input_size = 20
output_size = 30
learning_rate = 0.001
num_epochs = 50
patience = 5  # Early stopping patience

# Loss function
criterion = nn.MSELoss()

# Loop through each unique ID to train and save models
for obj_id, (data_2d, data_3d) in data_by_id.items():
    dataset = torch.utils.data.TensorDataset(data_2d, data_3d)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size

    if train_size == 0 or val_size == 0:
        print(f"Skipping ID {obj_id} due to insufficient data.")
        continue

    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

    model = PoseEstimationNet(input_size=input_size, output_size=output_size)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"\nStarting training for ID {obj_id}")
    train_model_for_id(
        model, train_loader, val_loader, criterion, optimizer, 
        num_epochs=num_epochs, patience=patience, device='cpu', obj_id=obj_id
    )


In [ ]:
import torch
import pandas as pd
import os

base_path = r"C:\Users\sakar\OneDrive\mt-datas\yolo\pose_estimation"
filename = "1_realistic_chair_train_2_yolo_result.csv"

# Construct paths to the data directories
data_2d_sample_path = os.path.join(base_path, filename)

# Define the function to load the trained model and use it for inference
def load_model_and_predict_3d(data_2d_path, model_path="sye0.pth", input_size=20, output_size=30):
    # Instantiate the model architecture and load weights
    model = PoseEstimationNet(input_size=input_size, output_size=output_size)
    model.load_state_dict(torch.load(model_path, weights_only=True))  # Added `weights_only=True`
    model.eval()  # Set the model to evaluation mode

    # Load the 2D data for inference
    try:
        df_2d = pd.read_csv(data_2d_path, header=None)
        df_2d = df_2d.iloc[:, 1:]  # Drop the ID column
        data_2d_tensor = torch.tensor(df_2d.values, dtype=torch.float32)
    except Exception as e:
        print(f"Error loading or processing 2D data: {e}")
        return None

    # Run the model to predict 3D points
    with torch.no_grad():
        predictions_3d = model(data_2d_tensor)
    
    print(predictions_3d)
    return predictions_3d

predictions_3d = load_model_and_predict_3d(data_2d_sample_path)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Function to plot 3D corner points from predictions and connect them to form a cube
def plot_3d_corners(predictions_3d):
    # Check if predictions are available
    if predictions_3d is None:
        print("No predictions to plot.")
        return
    
    # Extract the last 24 elements for the 8 corner points (each corner has 3 coordinates)
    corner_predictions = predictions_3d[:, -24:].view(-1, 8, 3)

    # Plot each object’s corners in a 3D space
    for i, corners in enumerate(corner_predictions):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        
        # Coordinates of the corners
        x, y, z = corners[:, 0], corners[:, 1], corners[:, 2]
        ax.scatter(x, y, z, c='b', marker='o')

        # Annotate each corner for clarity
        for j, (x_coord, y_coord, z_coord) in enumerate(corners):
            ax.text(x_coord, y_coord, z_coord, f"{j}", color="red")

        # Define connections for the cube
        connections = [
            (0, 1), (1, 2), (2, 3), (3, 0),  # Bottom square
            (4, 5), (5, 6), (6, 7), (7, 4),  # Top square
            (0, 4), (1, 5), (2, 6), (3, 7)   # Vertical edges connecting squares
        ]

        # Plot lines for each connection
        for start, end in connections:
            ax.plot(
                [x[start], x[end]],
                [y[start], y[end]],
                [z[start], z[end]],
                'b-'
            )

        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        plt.title(f"3D Corner Points and Cube Structure for Object")
        plt.show()

# Plot the 3D corners from the predictions
plot_3d_corners(predictions_3d)


# To Validate Results with Actual Data

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the points as a list of tuples (x, y, z)
points = [
    (3.049782, -1.4, -4.28893),  # Origin
    (3.404064, -0.2259196, -3.91012),
    (3.404064, -0.2259196, -4.593021),
    (2.711343, -0.2259196, -4.593021),
    (2.711343, -0.2259196, -3.91012),
    (3.404064, -1.4, -3.91012),
    (3.404064, -1.4, -4.593021),
    (2.711343, -1.4, -4.593021),
    (2.711343, -1.4, -3.91012)
]

# Define the edges of the cube by connecting the vertices
edges = [
    (1, 2), (2, 3), (3, 4), (4, 1),  # Top square
    (5, 6), (6, 7), (7, 8), (8, 5),  # Bottom square
    (1, 5), (2, 6), (3, 7), (4, 8)   # Vertical connections
]

# Create a figure with two subplots side by side
fig = plt.figure(figsize=(14, 6))

# Original 3D cube visualization
ax1 = fig.add_subplot(121, projection='3d')
for i, (x, y, z) in enumerate(points):
    if i == 0:
        ax1.scatter(x, y, z, color='red', s=100, label="Origin")  # Larger, red point for origin
        ax1.text(x, y, z, "Origin", color='red')
    else:
        ax1.scatter(x, y, z, color='blue', s=50)  # Blue points for other points
        ax1.text(x, y, z, f"{i-1}", color='blue')  # Label from 0 to 7

# Draw the edges for the original cube
for start, end in edges:
    x_line = [points[start][0], points[end][0]]
    y_line = [points[start][1], points[end][1]]
    z_line = [points[start][2], points[end][2]]
    ax1.plot(x_line, y_line, z_line, color="black")

# Setting labels and title for the original plot
ax1.set_xlabel("X Axis")
ax1.set_ylabel("Y Axis")
ax1.set_zlabel("Z Axis")
ax1.set_title("Original 3D Cube Visualization")

# Swapped axes visualization
ax2 = fig.add_subplot(122, projection='3d')
points_swapped = [(z, y, x) for x, y, z in points]  # Swap x and z values
for i, (x, y, z) in enumerate(points_swapped):
    if i == 0:
        ax2.scatter(x, y, z, color='red', s=100, label="Origin")  # Larger, red point for origin
        ax2.text(x, y, z, "Origin", color='red')
    else:
        ax2.scatter(x, y, z, color='blue', s=50)  # Blue points for other points
        ax2.text(x, y, z, f"{i-1}", color='blue')  # Label from 0 to 7

# Draw the edges for the swapped cube
for start, end in edges:
    x_line = [points_swapped[start][0], points_swapped[end][0]]
    y_line = [points_swapped[start][1], points_swapped[end][1]]
    z_line = [points_swapped[start][2], points_swapped[end][2]]
    ax2.plot(x_line, y_line, z_line, color="black")

# Setting labels and title for the swapped axes plot
ax2.set_xlabel("X Axis (original Z)")
ax2.set_ylabel("Y Axis")
ax2.set_zlabel("Z Axis (original X)")
ax2.set_title("3D Cube Visualization with Swapped Axes")

# Show the combined plot with a legend
plt.legend()
plt.show()


In [ ]:
import os
import pandas as pd
import numpy as np

# Path to the directories
base_path_2d = r"C:\Users\sakar\OneDrive\mt-datas\synthetic_data\8_correct_relative\2d_data"
base_path_3d = r"C:\Users\sakar\OneDrive\mt-datas\synthetic_data\8_correct_relative\3d_data"

# Functions to calculate errors
def calculate_mse(predicted, actual):
    return np.mean((predicted - actual) ** 2)

def calculate_rmse(predicted, actual):
    return np.sqrt(calculate_mse(predicted, actual))

def calculate_mae(predicted, actual):
    return np.mean(np.abs(predicted - actual))

# Loop through the 2D files
errors = {}
for file_name in os.listdir(base_path_2d):
    if file_name.endswith(".csv"):
        file_2d_path = os.path.join(base_path_2d, file_name)
        file_3d_path = os.path.join(base_path_3d, file_name)

        # Check if the corresponding 3D file exists
        if not os.path.exists(file_3d_path):
            print(f"3D data file for {file_name} not found. Skipping.")
            continue

        # Predict 3D points using the provided function
        predicted_3d = load_model_and_predict_3d(file_2d_path)
        if predicted_3d is None:
            print(f"Prediction failed for {file_name}. Skipping.")
            continue

        # Read the actual 3D points
        try:
            actual_3d_df = pd.read_csv(file_3d_path, header=None)
            actual_3d = actual_3d_df.iloc[:, 1:].values  # Drop the ID column
        except Exception as e:
            print(f"Error loading or processing 3D data for {file_name}: {e}")
            continue

        # Initialize error tracking for this file
        position_rmse, rotation_rmse, corner_rmse = [], [], []
        position_mae, rotation_mae, corner_mae = [], [], []
        total_object_rmse, total_object_mae = [], []

        # Loop through each row (object) in the file
        for i in range(predicted_3d.shape[0]):
            pred = predicted_3d[i]
            actual = actual_3d[i]

            # Ensure conversion to NumPy for each row
            if isinstance(pred, torch.Tensor):
                pred = pred.numpy()
            if isinstance(actual, torch.Tensor):
                actual = actual.numpy()

            # Extract components
            pred_position = pred[:3]
            actual_position = actual[:3]

            pred_rotation = pred[3:6]
            actual_rotation = actual[3:6]

            pred_corners = pred[6:].reshape(8, 3)
            actual_corners = actual[6:].reshape(8, 3)

            # Calculate RMSE and MAE for position, rotation, and corners
            pos_rmse = calculate_rmse(pred_position, actual_position)
            rot_rmse = calculate_rmse(pred_rotation, actual_rotation)
            corner_rmse_values = [calculate_rmse(pred_corners[j], actual_corners[j]) for j in range(8)]
            
            pos_mae = calculate_mae(pred_position, actual_position)
            rot_mae = calculate_mae(pred_rotation, actual_rotation)
            corner_mae_values = [calculate_mae(pred_corners[j], actual_corners[j]) for j in range(8)]

            # Total RMSE and MAE for this object
            object_rmse = pos_rmse + rot_rmse + sum(corner_rmse_values)
            object_mae = pos_mae + rot_mae + sum(corner_mae_values)

            # Append errors
            position_rmse.append(pos_rmse)
            rotation_rmse.append(rot_rmse)
            corner_rmse.append(sum(corner_rmse_values) / 8)  # Avg corner RMSE
            
            position_mae.append(pos_mae)
            rotation_mae.append(rot_mae)
            corner_mae.append(sum(corner_mae_values) / 8)  # Avg corner MAE
            
            total_object_rmse.append(object_rmse)
            total_object_mae.append(object_mae)

        # Aggregate RMSE and MAE for the file
        file_rmse = {
            "position": np.mean(position_rmse),
            "rotation": np.mean(rotation_rmse),
            "corners": np.mean(corner_rmse),
            "total": np.mean(total_object_rmse),
        }
        file_mae = {
            "position": np.mean(position_mae),
            "rotation": np.mean(rotation_mae),
            "corners": np.mean(corner_mae),
            "total": np.mean(total_object_mae),
        }

        # Save results
        errors[file_name] = {
            "RMSE": file_rmse,
            "MAE": file_mae,
        }

        print(f"Errors for {file_name}:\nRMSE: {file_rmse}\nMAE: {file_mae}")

# Display all errors
errors


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def visualize_errors_one_file(file_name, predicted_3d, actual_3d):
    if file_name not in errors:
        print(f"No errors found for {file_name}.")
        return

    # Extract RMSE and MAE for the file
    file_rmse = errors[file_name]["RMSE"]
    file_mae = errors[file_name]["MAE"]

    # Plot RMSE and MAE as bar charts
    labels = ["Position", "Rotation", "Corners", "Total"]
    rmse_values = [file_rmse["position"], file_rmse["rotation"], file_rmse["corners"], file_rmse["total"]]
    mae_values = [file_mae["position"], file_mae["rotation"], file_mae["corners"], file_mae["total"]]

    x = np.arange(len(labels))
    width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - width / 2, rmse_values, width, label="RMSE")
    plt.bar(x + width / 2, mae_values, width, label="MAE")

    plt.xlabel("Error Components")
    plt.ylabel("Error Value")
    plt.title(f"Error Metrics for {file_name}")
    plt.xticks(x, labels)
    plt.legend()
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    # 3D scatter plot for predicted vs. actual 3D points
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection="3d")

    for i in range(predicted_3d.shape[0]):
        pred = predicted_3d[i]
        actual = actual_3d[i]

        pred_corners = pred[6:].reshape(8, 3)
        actual_corners = actual[6:].reshape(8, 3)

        # Plot predicted corners in blue
        ax.scatter(pred_corners[:, 0], pred_corners[:, 1], pred_corners[:, 2], color="blue", label="Predicted" if i == 0 else "")
        # Plot actual corners in green
        ax.scatter(actual_corners[:, 0], actual_corners[:, 1], actual_corners[:, 2], color="green", label="Actual" if i == 0 else "")

    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_title(f"3D Points Comparison for {file_name}")
    ax.legend()
    plt.tight_layout()
    plt.show()

file_name = "2450.csv"
predicted_3d = load_model_and_predict_3d(os.path.join(base_path_2d, file_name))
actual_3d_df = pd.read_csv(os.path.join(base_path_3d, file_name), header=None)
actual_3d = actual_3d_df.iloc[:, 1:].values

visualize_errors_one_file(file_name, predicted_3d, actual_3d)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def visualize_errors_all_files_stacked(errors):
    # Extract errors into DataFrame
    data = []
    for file_name, metrics in errors.items():
        rmse = metrics["RMSE"]
        mae = metrics["MAE"]
        data.append([
            file_name,
            rmse["position"], rmse["rotation"], rmse["corners"],
            mae["position"], mae["rotation"], mae["corners"]
        ])

    columns = ["File", "RMSE_Position", "RMSE_Rotation", "RMSE_Corners",
               "MAE_Position", "MAE_Rotation", "MAE_Corners"]
    df = pd.DataFrame(data, columns=columns)

    x = np.arange(len(df))
    
    # Generate plots
    # 4 for MAE
    plt.figure(figsize=(12, 8))
    plt.bar(x, df["MAE_Corners"], color="lightgreen", label="Corners")
    plt.xlabel("CSV Files")
    plt.ylabel("MAE")
    plt.title("MAE Errors for Corners")
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.bar(x, df["MAE_Position"], color="skyblue", label="Position")
    plt.xlabel("CSV Files")
    plt.ylabel("MAE")
    plt.title("MAE Errors for Position")
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.bar(x, df["MAE_Rotation"], color="orange", label="Rotation")
    plt.xlabel("CSV Files")
    plt.ylabel("MAE")
    plt.title("MAE Errors for Rotation")
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.bar(x, df["MAE_Position"], color="skyblue", label="Position")
    plt.bar(x, df["RMSE_Corners"], bottom=df["RMSE_Position"] , color="lightgreen", label="Corners")
    plt.bar(x, df["RMSE_Rotation"], bottom=df["RMSE_Position"]+ df["RMSE_Corners"], color="orange", label="Rotation")
    plt.xlabel("CSV Files")
    plt.ylabel("MAE")
    plt.title("MAE Total Errors (Stacked)")
    plt.legend()
    plt.show()

    # 4 for RMSE
    plt.figure(figsize=(12, 8))
    plt.bar(x, df["RMSE_Corners"], color="lightgreen", label="Corners")
    plt.xlabel("CSV Files")
    plt.ylabel("RMSE")
    plt.title("RMSE Errors for Corners")
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.bar(x, df["RMSE_Position"], color="skyblue", label="Position")
    plt.xlabel("CSV Files")
    plt.ylabel("RMSE")
    plt.title("RMSE Errors for Position")
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.bar(x, df["RMSE_Rotation"], color="orange", label="Rotation")
    plt.xlabel("CSV Files")
    plt.ylabel("RMSE")
    plt.title("RMSE Errors for Rotation")
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.bar(x, df["RMSE_Position"], color="skyblue", label="Position")
    plt.bar(x, df["RMSE_Corners"], bottom=df["RMSE_Position"] , color="lightgreen", label="Corners")
    plt.bar(x, df["RMSE_Rotation"], bottom=df["RMSE_Position"]+ df["RMSE_Corners"], color="orange", label="Rotation")
    plt.xlabel("CSV Files")
    plt.ylabel("RMSE")
    plt.title("RMSE Total Errors (Stacked)")
    plt.legend()
    plt.show()

# Example: Visualize stacked RMSE and MAE errors across all files
visualize_errors_all_files_stacked(errors)


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

# Randomly select 10 files from the errors dictionary
selected_files = random.sample(list(errors.keys()), 10)

# Extract the selected errors into a new dictionary
selected_errors = {file: errors[file] for file in selected_files}

print("Selected Files for Visualization:")
print(selected_files)

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def visualize_selected_errors_stacked(selected_errors):
    # Extract errors into DataFrame
    data = []
    for file_name, metrics in selected_errors.items():
        rmse = metrics["RMSE"]
        mae = metrics["MAE"]
        data.append([
            file_name,
            rmse["position"], rmse["rotation"], rmse["corners"],
            mae["position"], mae["rotation"], mae["corners"]
        ])

    columns = ["File", "RMSE_Position", "RMSE_Rotation", "RMSE_Corners","MAE_Position", "MAE_Rotation", "MAE_Corners"]
    df = pd.DataFrame(data, columns=columns)

    x = np.arange(len(df))
    
    # 4 for MAE
    plt.figure(figsize=(12, 8))
    plt.bar(x, df["MAE_Corners"], color="lightgreen", label="Corners")
    plt.xticks(x, df["File"], rotation=45, ha="right")
    plt.xlabel("CSV Files")
    plt.ylabel("MAE")
    plt.title("MAE Errors for Corners")
    plt.legend()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.bar(x, df["MAE_Position"], color="skyblue", label="Position")
    plt.xticks(x, df["File"], rotation=45, ha="right")
    plt.xlabel("CSV Files")
    plt.ylabel("MAE")
    plt.title("MAE Errors for Position")
    plt.legend()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.bar(x, df["MAE_Rotation"], color="orange", label="Rotation")
    plt.xticks(x, df["File"], rotation=45, ha="right")
    plt.xlabel("CSV Files")
    plt.ylabel("MAE")
    plt.title("MAE Errors for Rotation")
    plt.legend()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.bar(x, df["MAE_Position"], color="skyblue", label="Position")
    plt.bar(x, df["MAE_Corners"], bottom=df["MAE_Position"], color="lightgreen", label="Corners")
    plt.bar(x, df["MAE_Rotation"], bottom=df["MAE_Position"] + df["MAE_Corners"], color="orange", label="Rotation")
    
    plt.xticks(x, df["File"], rotation=45, ha="right")
    plt.xlabel("CSV Files")
    plt.ylabel("MAE")
    plt.title("MAE Total Errors (Stacked)")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # 4 for RMSE
    plt.figure(figsize=(12, 8))
    plt.bar(x, df["RMSE_Corners"], color="lightgreen", label="Corners")
    plt.xticks(x, df["File"], rotation=45, ha="right")
    plt.xlabel("CSV Files")
    plt.ylabel("RMSE")
    plt.title("RMSE Errors for Corners")
    plt.legend()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.bar(x, df["RMSE_Position"], color="skyblue", label="Position")
    plt.xticks(x, df["File"], rotation=45, ha="right")
    plt.xlabel("CSV Files")
    plt.ylabel("RMSE")
    plt.title("RMSE Errors for Position")
    plt.legend()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.bar(x, df["RMSE_Rotation"], color="orange", label="Rotation")
    plt.xticks(x, df["File"], rotation=45, ha="right")
    plt.xlabel("CSV Files")
    plt.ylabel("RMSE")
    plt.title("RMSE Errors for Rotation")
    plt.legend()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 8))
    plt.bar(x, df["RMSE_Position"], color="skyblue", label="Position")
    plt.bar(x, df["MAE_Corners"], bottom=df["MAE_Position"], color="lightgreen", label="Corners")
    plt.bar(x, df["MAE_Rotation"], bottom=df["MAE_Position"] + df["MAE_Corners"], color="orange", label="Rotation")
    plt.xticks(x, df["File"], rotation=45, ha="right")
    plt.xlabel("CSV Files")
    plt.ylabel("RMSE")
    plt.title("RMSE Total Errors (Stacked)")
    plt.legend()
    plt.tight_layout()
    plt.show()

# Visualize the errors for the selected files
visualize_selected_errors_stacked(selected_errors)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Function to identify outliers and median
def identify_outliers_and_median(errors, category):
    data = []
    for file_name, metrics in errors.items():
        rmse = metrics["RMSE"]
        mae = metrics["MAE"]
        data.append([
            file_name,
            rmse["position"], rmse["rotation"], rmse["corners"],
            mae["position"], mae["rotation"], mae["corners"],
            rmse["position"] + rmse["rotation"] + rmse["corners"],  # Total RMSE
            mae["position"] + mae["rotation"] + mae["corners"],    # Total MAE
        ])
    
    columns = ["File", "RMSE_Position", "RMSE_Rotation", "RMSE_Corners",
               "MAE_Position", "MAE_Rotation", "MAE_Corners",
               "RMSE_Total", "MAE_Total"]
    df = pd.DataFrame(data, columns=columns)

    # Get top 10 highest, lowest, and median for the selected category
    top_10_highest = df.nlargest(10, category)
    top_10_lowest = df.nsmallest(10, category)
    median_start = len(df) // 2 - 5
    median_end = median_start + 10
    median_10 = df.sort_values(by=category).iloc[median_start:median_end]

    return top_10_highest, top_10_lowest, median_10

# Function to visualize outliers and median
def visualize_outliers_and_median(top_10_high, top_10_low, median_10, category, metric_name):
    # Combine high, median, and low into a single DataFrame for plotting
    combined = pd.concat([top_10_high, median_10, top_10_low])
    labels = combined["File"]
    values = combined[category]

    # Colors: Orange for highest, Green for median, Skyblue for lowest
    colors = (
        ['orange'] * len(top_10_high) +
        ['green'] * len(median_10) +
        ['skyblue'] * len(top_10_low)
    )

    # Bar Plot
    plt.figure(figsize=(14, 8))
    bars = plt.bar(labels, values, color=colors)

    # Create a custom legend
    legend_handles = [
        bars[0],  # Example bar for highest (orange)
        bars[len(top_10_high)],  # Example bar for median (green)
        bars[-1],  # Example bar for lowest (skyblue)
    ]
    legend_labels = ["Top 10 Highest", "10 Median", "Top 10 Lowest"]

    # Plot customization
    plt.xlabel("CSV Files")
    plt.ylabel(metric_name)
    plt.title(f"Outliers and Median for {metric_name} - {category}")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.legend(legend_handles, legend_labels)
    plt.show()

# Loop through all error categories including totals and generate plots
categories = [
    "MAE_Position", "MAE_Rotation", "MAE_Corners", "MAE_Total",
    "RMSE_Position", "RMSE_Rotation", "RMSE_Corners", "RMSE_Total"
]

for category in categories:
    metric_name = "MAE" if "MAE" in category else "RMSE"
    print(f"\nProcessing Outliers and Median for {category}")
    
    # Identify outliers and median
    top_10_high, top_10_low, median_10 = identify_outliers_and_median(errors, category)
    
    # Visualize outliers and median
    visualize_outliers_and_median(top_10_high, top_10_low, median_10, category, metric_name)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Input coordinates (X and Z components)
points_xz = np.array([
    [-3.1509, 5.7967],  # P0
    [-3.0486, 5.2867],  # P1
    [-4.3261, 4.4202],  # P2
    [-4.3691, 4.9874]   # P3
])

# Calculate the vectors
vec_p1_p0 = points_xz[0] - points_xz[1]  # P1->P0
vec_p2_p1 = points_xz[1] - points_xz[2]  # P2->P1

# Function to calculate angle with a given axis
def calculate_angle(vector, axis):
    cos_theta = np.dot(vector, axis) / np.linalg.norm(vector)  # Cosine of the angle
    angle_radians = np.arccos(cos_theta)  # Angle in radians
    # Determine the direction of the angle (clockwise or counterclockwise)
    cross_product = vector[0] * axis[1] - vector[1] * axis[0]  # 2D cross product scalar
    if cross_product < 0:  # Negative cross product means clockwise
        angle_radians = -angle_radians
    return angle_radians

# Calculate angles
z_axis = np.array([0, 1])  # Z-axis in the X-Z plane
x_axis = np.array([1, 0])  # X-axis in the X-Z plane

angle_radians_p1_p0 = calculate_angle(vec_p1_p0, z_axis)  # Align P1->P0 with Z-axis
angle_radians_p2_p1 = calculate_angle(vec_p2_p1, x_axis)  # Align P2->P1 with X-axis

# Visualization
plt.figure(figsize=(8, 8))

# Plot the points
for i, point in enumerate(points_xz):
    plt.scatter(point[0], point[1], label=f"P{i}")
    plt.text(point[0], point[1], f"P{i}", fontsize=9)

# Plot the vectors
plt.quiver(points_xz[1, 0], points_xz[1, 1], vec_p1_p0[0], vec_p1_p0[1], angles='xy', scale_units='xy', scale=1, color='b', label="P1->P0")
plt.quiver(points_xz[2, 0], points_xz[2, 1], vec_p2_p1[0], vec_p2_p1[1], angles='xy', scale_units='xy', scale=1, color='g', label="P2->P1")

# Add axis lines
plt.axhline(0, color='gray', linewidth=0.8)
plt.axvline(0, color='gray', linewidth=0.8)

# Set labels and title
plt.xlabel("X (Right)")
plt.ylabel("Z (Forward)")
plt.title("2D Visualization on X-Z Plane (Aligning P1->P0 to Z-axis and P2->P1 to X-axis)")
plt.legend()
plt.grid()
plt.axis('equal')  # Keep proportions equal
plt.show()

# Print results
print("Vector P1->P0:", vec_p1_p0)
print("Rotation Angle (radians) to align P1->P0 with Z-axis:", angle_radians_p1_p0)
print("Rotation Angle (degrees) to align P1->P0 with Z-axis:", np.degrees(angle_radians_p1_p0))
print("Vector P2->P1:", vec_p2_p1)
print("Rotation Angle (radians) to align P2->P1 with X-axis:", angle_radians_p2_p1)
print("Rotation Angle (degrees) to align P2->P1 with X-axis:", np.degrees(angle_radians_p2_p1))
